In [ ]:
import random
import glob
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import keras

In [ ]:
model = keras.models.load_model('GarbageModel.h5')

In [ ]:
FILTER_SIZE = 32 #Size of filter used for heatmap
IMG_SIZE = 224
label_to_index = {'Normal': 0, 'People': 1, 'Potential Pileup': 2, 'Pileup': 3}
index_to_label = {v: k for k, v in label_to_index.items()}

In [ ]:
#loads an image from a path
def loadImage(image_path):
    image = Image.open(image_path)
    image = np.array((image.resize((IMG_SIZE, IMG_SIZE)))) / 255.0
    image = np.reshape(image, [1,IMG_SIZE, IMG_SIZE, 3])
    return image

In [ ]:
#creates a heatmap for an image.
#Accomplishes this by convolving a 0 box of size FILTER_SIZE around the image.
#At each point, the model is evaluated and the probabilities of each class are added to the corresponding heatmap.
#If the probability drops when the 0 box is over that part of the image, it can be reasoned that the classifier sees
#those pixels as important to the classification it has made.
def getHeatMap(image_path):
    def getFilterImage(image,x,y):
        #returns an image for the filter applied at (x,y)
        image = np.array(image, copy=True) #make a new copy
        for i in range(FILTER_SIZE):
            for j in range(FILTER_SIZE):
                image[0][i+x][j+y] = 0
        return image
    
    heatMapSize = int(IMG_SIZE/FILTER_SIZE)
    heatMaps = [np.zeros((heatMapSize, heatMapSize)) for x in range(4)] #creates list with heatmaps for each label
    image = loadImage(image_path)
    labelIndex = 0
    for i in range(heatMapSize):
        for j in range(heatMapSize):
            img = getFilterImage(image, i*FILTER_SIZE, j*FILTER_SIZE)
            scores = model.predict(img)
            for k, heatMap in enumerate(heatMaps):
                heatMap[i][j] = scores[0][k]
    for heatMap in heatMaps:
        heatMap = heatMap/np.amax(heatMap)
    return heatMaps

In [ ]:
#shows heatmaps for all classes
def allHeatMaps(image_path, heatMaps, labelIndex):
    fig = plt.figure(figsize=(10,10))
    ax1 =fig.add_subplot(2,3,1)
    ax1.axis('off')
    ax1.title.set_text('Original Image')
    image = loadImage(image_path)
    ax1.imshow(np.reshape(image, (IMG_SIZE,IMG_SIZE,3)))
    for i, heatMap in enumerate(heatMaps):
        ax = fig.add_subplot(2, 3 , i+2)
        ax.axis('off')
        ax.title.set_text(index_to_label[i] + " Heatmap")
        ax.imshow(heatMaps[i])
    img_name = image_path.split('\\')[-1].split('.')[0]
    plt.savefig("HeatMaps\\" + index_to_label[labelIndex] + "\\" + img_name + 'all.jpg')
    plt.close("all")

In [ ]:
#creates heatmap for the only the images true class
def singleHeatMap(heatMap, image_path, labelIndex):
    fig = plt.figure(figsize=(10,10))
    ax1 = fig.add_subplot(1,2,1)
    ax1.axis('off')
    ax1.title.set_text('Original Image')
    image = loadImage(image_path)
    ax1.imshow(np.reshape(image, (IMG_SIZE,IMG_SIZE,3)))
    ax2 = fig.add_subplot(1,2,2)
    ax2.axis('off')
    ax2.title.set_text(index_to_label[labelIndex] + " Heatmap")
    ax2.imshow(heatMap)
    img_name = image_path.split('\\')[-1].split('.')[0]
    plt.savefig("HeatMaps\\" + index_to_label[labelIndex] + "\\" + img_name + 'single.jpg')
    plt.close("all")

In [ ]:
#how many heatmaps of each class are to be created
normal = 4
people = 4
potentialPileup = 4
pileup = 4

In [ ]:
#loads image paths randomly from each folder
def getImageList(path, length):
    image_paths = glob.glob(path)
    random.shuffle(image_paths)
    return image_paths[:length]

image_paths = getImageList("CompactorLabeled\\Normal\\*.jpg", normal)
image_paths += getImageList("CompactorLabeled\\People\\*.jpg", people)
image_paths += getImageList("CompactorLabeled\\PotentialPileup\\*.jpg", potentialPileup)
image_paths += getImageList("CompactorLabeled\\Pileup\\*.jpg", pileup)
image_labels = [0] * normal + [1] * people + [2] * potentialPileup + [3] * pileup

In [ ]:
#generates all heat maps in folders. This takes a long time to complete.
#Make sure there is a HeatMaps/LabelName directory present for each LabelName
for i, image_path in enumerate(image_paths):
    heatMaps = getHeatMap(image_path)
    allHeatMaps(image_path, heatMaps, image_labels[i])
    singleHeatMap(heatMaps[image_labels[i]], image_path, image_labels[i])